In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Math

In [5]:
# Hệ số M đủ lớn ta sẽ dùng trong phương pháp Big-M ở bài này
M = 1000

# Đọc và chuẩn bị dữ liệu đầu vào
def parse_input():
    """
    Hàm này định nghĩa bài toán tối ưu tuyến tính.
    
    Returns:
        tuple: (c, A, b, signs) - Hệ số hàm mục tiêu, ma trận hệ số, vế phải, dấu ràng buộc
    """
    # Bài toán tìm Max hàm f = 2500x1 + 2000x2
    # Các ràng buộc:
    # x1 + x2 <= 40
    # 150x1 + 250x2 <= 9000
    # 1000x1 + 1500x2 <= 50000
    # 2x1 - x2 <= 0
    c = [2500, 2000]
    A = [
        [1, 1],
        [150, 250],
        [1000, 1500],
        [2, -1]
    ]
    b = [40, 9000, 50000, 0]
    signs = ['<=', '<=', '<=', '<=']
    return c, A, b, signs


In [6]:
# Hiển thị bài toán
def display_problem(c, A, b, signs):
    """
    Hiển thị bài toán tối ưu tuyến tính dưới dạng dễ đọc
    """
    n = len(c)
    m = len(b)
    
    # Hiển thị hàm mục tiêu
    obj_func = "Max Z = "
    for i in range(n):
        if i > 0:
            if c[i] >= 0:
                obj_func += f" + {c[i]}x{i+1}" if c[i] != 1 else f" + x{i+1}"
            else:
                obj_func += f" - {abs(c[i])}x{i+1}" if abs(c[i]) != 1 else f" - x{i+1}"
        else:
            obj_func += f"{c[i]}x{i+1}" if c[i] != 1 else f"x{i+1}"
    
    # Hiển thị các ràng buộc
    constraints = ["Subject to:"]
    for i in range(m):
        constraint = ""
        for j in range(n):
            if j > 0:
                if A[i][j] >= 0:
                    constraint += f" + {A[i][j]}x{j+1}" if A[i][j] != 1 else f" + x{j+1}"
                else:
                    constraint += f" - {abs(A[i][j])}x{j+1}" if abs(A[i][j]) != 1 else f" - x{j+1}"
            else:
                if A[i][j] >= 0:
                    constraint += f"{A[i][j]}x{j+1}" if A[i][j] != 1 else f"x{j+1}"
                else:
                    constraint += f"-{abs(A[i][j])}x{j+1}" if abs(A[i][j]) != 1 else f"-x{j+1}"
        
        constraint += f" {signs[i]} {b[i]}"
        constraints.append(constraint)
    
    # Hiển thị ràng buộc không âm
    non_neg = "x_j ≥ 0, j = 1,2,...," + str(n)
    
    # In ra kết quả
    display(Markdown(f"**Bài toán tối ưu tuyến tính:**"))
    display(Markdown(obj_func))
    for constraint in constraints:
        display(Markdown(constraint))
    display(Markdown(non_neg))

# Test hiển thị bài toán
c, A, b, signs = parse_input()
display_problem(c, A, b, signs)

**Bài toán tối ưu tuyến tính:**

Max Z = 2500x1 + 2000x2

Subject to:

x1 + x2 <= 40

150x1 + 250x2 <= 9000

1000x1 + 1500x2 <= 50000

2x1 - x2 <= 0

x_j ≥ 0, j = 1,2,...,2

In [7]:
# Tiền xử lý các ràng buộc

def preprocess_constraints(A, b, signs):
    """
    Tiền xử lý các ràng buộc để đảm bảo vế phải không âm.
    
    Args:
        A (list): Ma trận hệ số của các ràng buộc
        b (list): Vecto vế phải
        signs (list): Danh sách các dấu ràng buộc ('<=', '=', '>=')
        
    Returns:
        tuple: (new_A, new_b, new_signs) - Ma trận hệ số, vecto vế phải và dấu ràng buộc sau khi xử lý
    """
    new_A = []
    new_b = []
    new_signs = []

    for i in range(len(A)):
        row = A[i].copy()  # Tạo bản sao để tránh thay đổi dữ liệu gốc
        rhs = b[i]
        sign = signs[i]

        # Đưa RHS về không âm
        if rhs < 0:
            row = [-x for x in row]
            rhs = -rhs
            if sign == '<=':
                sign = '>='
            elif sign == '>=':
                sign = '<='

        new_A.append(row)
        new_b.append(rhs)
        new_signs.append(sign)

    return new_A, new_b, new_signs

# Test preprocessing
new_A, new_b, new_signs = preprocess_constraints(A, b, signs)

# Hiển thị kết quả preprocessing
print("=== Sau khi tiền xử lý ===")
for i in range(len(new_A)):
    row_str = " + ".join([f"{coef}x{j+1}" if j == 0 or coef >= 0 else f"- {abs(coef)}x{j+1}" 
                           for j, coef in enumerate(new_A[i]) if coef != 0])
    print(f"{row_str} {new_signs[i]} {new_b[i]}")

=== Sau khi tiền xử lý ===
1x1 + 1x2 <= 40
150x1 + 250x2 <= 9000
1000x1 + 1500x2 <= 50000
2x1 + - 1x2 <= 0


In [8]:
# Xây dựng bảng đơn hình Big-M

def build_bigM_tableau(c, A, b, signs):
    """
    Xây dựng bảng đơn hình ban đầu cho phương pháp Big-M.
    
    Args:
        c (list): Hệ số hàm mục tiêu
        A (list): Ma trận hệ số của các ràng buộc
        b (list): Vecto vế phải
        signs (list): Danh sách các dấu ràng buộc ('<=', '=', '>=')
        
    Returns:
        tuple: (basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)
    """
    A, b, signs = preprocess_constraints(A, b, signs)

    num_orig_vars = len(c)
    tableau = []
    basis = []
    basis_cost = []
    var_names = [f"x{i+1}" for i in range(num_orig_vars)]
    c_extended = c[:]  # Bắt đầu với hệ số gốc

    for i, (row, sign) in enumerate(zip(A, signs)):
        row_extended = row[:]

        if sign == '<=':
            # Thêm biến slack
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(0)
            basis.append(len(var_names) - 1)
            basis_cost.append(0)

        elif sign == '>=':
            # Thêm biến surplus và artificial
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(-1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(0)

            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(-M)
            basis.append(len(var_names) - 1)
            basis_cost.append(-M)

        elif sign == '=':
            # Thêm artificial
            row_extended += [0] * (len(var_names) - len(row_extended))
            row_extended.append(1)
            var_names.append(f"x{len(var_names)+1}")
            c_extended.append(-M)
            basis.append(len(var_names) - 1)
            basis_cost.append(-M)

        tableau.append(row_extended)

    # Đệm 0 để làm đều các dòng
    total_vars = len(var_names)
    for row in tableau:
        row += [0] * (total_vars - len(row))

    # Cách tính z_row dựa trên định nghĩa
    z_row = [0] * total_vars
    rhs_z = 0
    
    # Tính RHS của hàng Z: Σ(cBi·bi)
    for i in range(len(tableau)):
        rhs_z += basis_cost[i] * b[i]
    
    # Với mỗi biến j, tính Δj = Σ(cBi·aij) - cj
    for j in range(total_vars):
        # Tính Σ(cBi·aij)
        sum_cb_aij = 0
        for i in range(len(tableau)):
            sum_cb_aij += basis_cost[i] * tableau[i][j]
        
        # Trừ đi cj
        z_row[j] = sum_cb_aij - c_extended[j]

    return basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended

# Xây dựng bảng từ dữ liệu đầu vào
basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended = build_bigM_tableau(c, A, b, signs)

In [9]:
# Hiển thị bảng đơn hình Big-M

def print_bigM_tableau(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended):
    """
    Hiển thị bảng đơn hình cho phương pháp Big-M.
    
    Args:
        basis (list): Chỉ số của các biến trong cơ sở
        basis_cost (list): Chi phí của các biến trong cơ sở
        b (list): Vecto vế phải
        tableau (list): Bảng đơn hình
        z_row (list): Hàng Z-row (reduced costs)
        rhs_z (float): Giá trị hàm mục tiêu
        var_names (list): Tên của các biến
        c_extended (list): Hệ số chi phí mở rộng
    """
    print("\n=== Bảng đơn hình ban đầu (phương pháp Big-M) ===\n")

    num_vars = len(var_names)
    header = ["Biến cơ sở", "Hệ số", "Hệ số RHS"] + var_names
    print(" | ".join(f"{h:^12}" for h in header))
    print("-" * (14 * len(header)))

    # Hiển thị hệ số hàm mục tiêu (bao gồm cả artificial variables)
    c_row = [" " * 12, " " * 12, " " * 12] + [f"{coef:^12}" for coef in c_extended]
    print(" | ".join(c_row))
    print("-" * (14 * len(header)))

    for i in range(len(basis)):
        var = var_names[basis[i]]
        coef = basis_cost[i]
        row_vals = tableau[i]
        row_str = [f"{var:^12}", f"{coef:^12}", f"{b[i]:^12}"] + [f"{v:^12}" for v in row_vals]
        print(" | ".join(row_str))

    # Hiển thị toàn bộ z_row
    z_row_str = ["Z".center(12), " ".center(12), f"{rhs_z:^12}"] + [f"{v:^12}" for v in z_row]
    print("-" * (14 * len(header)))
    print(" | ".join(z_row_str))

# Hiển thị bảng đơn hình dạng Pandas DataFrame
def display_tableau_df(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended):
    """
    Hiển thị bảng đơn hình dưới dạng DataFrame cho dễ đọc.
    """
    # Tạo DataFrame cho bảng chính
    data = []
    for i in range(len(basis)):
        row_data = {
            'Biến cơ sở': var_names[basis[i]],
            'Hệ số': basis_cost[i],
            'RHS': b[i]
        }
        for j, name in enumerate(var_names):
            row_data[name] = tableau[i][j]
        data.append(row_data)
    
    # Thêm hàng Z
    z_data = {
        'Biến cơ sở': 'Z',
        'Hệ số': '',
        'RHS': rhs_z
    }
    for j, name in enumerate(var_names):
        z_data[name] = z_row[j]
    data.append(z_data)
    
    # Tạo DataFrame
    df = pd.DataFrame(data)
    
    # Hiển thị các hệ số hàm mục tiêu
    display(Markdown("**Hệ số hàm mục tiêu (c):**"))
    c_data = {}
    for j, name in enumerate(var_names):
        c_data[name] = c_extended[j]
    c_df = pd.DataFrame([c_data])
    display(c_df)
    
    display(Markdown("**Bảng đơn hình:**"))
    return df

# Hiển thị bảng đơn hình
print_bigM_tableau(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)

# Hiển thị bảng đơn hình dạng DataFrame
tableau_df = display_tableau_df(basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended)
display(tableau_df)


=== Bảng đơn hình ban đầu (phương pháp Big-M) ===

 Biến cơ sở  |    Hệ số     |  Hệ số RHS   |      x1      |      x2      |      x3      |      x4      |      x5      |      x6     
------------------------------------------------------------------------------------------------------------------------------
             |              |              |     2500     |     2000     |      0       |      0       |      0       |      0      
------------------------------------------------------------------------------------------------------------------------------
     x3      |      0       |      40      |      1       |      1       |      1       |      0       |      0       |      0      
     x4      |      0       |     9000     |     150      |     250      |      0       |      1       |      0       |      0      
     x5      |      0       |    50000     |     1000     |     1500     |      0       |      0       |      1       |      0      
     x6      |      0       |

**Hệ số hàm mục tiêu (c):**

,x1,x2,x3,x4,x5,x6
0,2500,2000,0,0,0,0


**Bảng đơn hình:**

,Biến cơ sở,Hệ số,RHS,x1,x2,x3,x4,x5,x6
0,x3,0,40,1,1,1,0,0,0
1,x4,0,9000,150,250,0,1,0,0
2,x5,0,50000,1000,1500,0,0,1,0
3,x6,0,0,2,-1,0,0,0,1
4,Z,,0,-2500,-2000,0,0,0,0


In [10]:
# Thuật toán đơn hình Big-M

def select_pivot_column(z_row):
    """
    Chọn cột pivot dựa trên giá trị reduced cost tích cực nhất.
    Đối với bài toán Max, chọn cột có giá trị âm nhất.
    
    Args:
        z_row (list): Hàng Z-row (reduced costs)
        
    Returns:
        int: Chỉ số của cột pivot
    """
    min_val = 0
    min_idx = -1
    
    for j, val in enumerate(z_row):
        if val < min_val:
            min_val = val
            min_idx = j
    
    return min_idx

def select_pivot_row(tableau, b, pivot_col):
    """
    Chọn hàng pivot dựa trên minimum ratio test.
    
    Args:
        tableau (list): Bảng đơn hình
        b (list): Vecto vế phải
        pivot_col (int): Chỉ số của cột pivot
        
    Returns:
        int: Chỉ số của hàng pivot
    """
    min_ratio = float('inf')
    min_idx = -1
    
    for i in range(len(tableau)):
        if tableau[i][pivot_col] > 0:
            ratio = b[i] / tableau[i][pivot_col]
            if ratio < min_ratio:
                min_ratio = ratio
                min_idx = i
    
    return min_idx

def pivot_operation(tableau, b, z_row, rhs_z, basis, basis_cost, pivot_row, pivot_col, c_extended):
    """
    Thực hiện phép pivot trên bảng đơn hình.
    
    Args:
        tableau (list): Bảng đơn hình
        b (list): Vecto vế phải
        z_row (list): Hàng Z-row
        rhs_z (float): Giá trị hàm mục tiêu
        basis (list): Chỉ số của các biến trong cơ sở
        basis_cost (list): Chi phí của các biến trong cơ sở
        pivot_row (int): Chỉ số của hàng pivot
        pivot_col (int): Chỉ số của cột pivot
        c_extended (list): Hệ số chi phí mở rộng
        
    Returns:
        tuple: (tableau, b, z_row, rhs_z, basis, basis_cost) - Bảng đơn hình sau khi pivot
    """
    # Cập nhật biến cơ sở
    basis[pivot_row] = pivot_col
    basis_cost[pivot_row] = c_extended[pivot_col]
    
    # Chia hàng pivot cho giá trị pivot
    pivot_value = tableau[pivot_row][pivot_col]
    b[pivot_row] /= pivot_value
    for j in range(len(tableau[pivot_row])):
        tableau[pivot_row][j] /= pivot_value
    
    # Cập nhật các hàng khác
    for i in range(len(tableau)):
        if i != pivot_row:
            factor = tableau[i][pivot_col]
            b[i] -= factor * b[pivot_row]
            for j in range(len(tableau[i])):
                tableau[i][j] -= factor * tableau[pivot_row][j]
    
    # Cập nhật Z-row và RHS_Z
    factor = z_row[pivot_col]
    rhs_z -= factor * b[pivot_row]
    for j in range(len(z_row)):
        z_row[j] -= factor * tableau[pivot_row][j]
    
    return tableau, b, z_row, rhs_z, basis, basis_cost

def solve_bigM(c, A, b, signs, max_iter=100):
    """
    Giải bài toán tối ưu tuyến tính bằng phương pháp Big-M.
    
    Args:
        c (list): Hệ số hàm mục tiêu
        A (list): Ma trận hệ số của các ràng buộc
        b (list): Vecto vế phải
        signs (list): Danh sách các dấu ràng buộc ('<=', '=', '>=')
        max_iter (int): Số lần lặp tối đa
        
    Returns:
        tuple: (optimal_value, optimal_solution, iterations, tableau_history)
            - optimal_value: Giá trị tối ưu của hàm mục tiêu
            - optimal_solution: Véc tơ nghiệm tối ưu
            - iterations: Số lần lặp thực hiện
            - tableau_history: Lịch sử các bảng đơn hình
    """
    # Xây dựng bảng đơn hình ban đầu
    basis, basis_cost, new_b, tableau, z_row, rhs_z, var_names, c_extended = build_bigM_tableau(c, A, b, signs)
    
    tableau_history = []
    iterations = 0
    
    # Lưu trạng thái ban đầu
    tableau_history.append({
        'iteration': iterations,
        'basis': basis.copy(),
        'basis_cost': basis_cost.copy(),
        'b': new_b.copy(),
        'tableau': [row.copy() for row in tableau],
        'z_row': z_row.copy(),
        'rhs_z': rhs_z,
        'pivot_row': None,
        'pivot_col': None
    })
    
    # Lặp cho đến khi đạt tối ưu hoặc đạt số lần lặp tối đa
    while iterations < max_iter:
        iterations += 1
        
        # Chọn cột pivot (biến vào)
        pivot_col = select_pivot_column(z_row)
        if pivot_col == -1:
            # Đã đạt tối ưu
            break
        
        # Chọn hàng pivot (biến ra)
        pivot_row = select_pivot_row(tableau, new_b, pivot_col)
        if pivot_row == -1:
            # Bài toán không giới nội
            return None, None, iterations, tableau_history
        
        # Thực hiện phép pivot
        tableau, new_b, z_row, rhs_z, basis, basis_cost = pivot_operation(
            tableau, new_b, z_row, rhs_z, basis, basis_cost, pivot_row, pivot_col, c_extended
        )
        
        # Lưu trạng thái hiện tại
        tableau_history.append({
            'iteration': iterations,
            'basis': basis.copy(),
            'basis_cost': basis_cost.copy(),
            'b': new_b.copy(),
            'tableau': [row.copy() for row in tableau],
            'z_row': z_row.copy(),
            'rhs_z': rhs_z,
            'pivot_row': pivot_row,
            'pivot_col': pivot_col
        })
    
    # Kiểm tra nếu vẫn có biến nhân tạo trong cơ sở
    has_artificial_in_basis = False
    for i, var_idx in enumerate(basis):
        if var_idx >= len(c) and basis_cost[i] == -M and new_b[i] > 1e-10:
            has_artificial_in_basis = True
            break
    
    if has_artificial_in_basis:
        # Bài toán vô nghiệm
        return None, None, iterations, tableau_history
    
    # Xây dựng nghiệm tối ưu
    optimal_solution = [0] * len(c)
    for i, var_idx in enumerate(basis):
        if var_idx < len(c):
            optimal_solution[var_idx] = new_b[i]
    
    # Tính giá trị tối ưu thực sự (không tính các biến nhân tạo)
    optimal_value = sum(c[j] * optimal_solution[j] for j in range(len(c)))
    
    return optimal_value, optimal_solution, iterations, tableau_history

# Giải bài toán
optimal_value, optimal_solution, iterations, tableau_history = solve_bigM(c, A, b, signs)

# Hiển thị kết quả
if optimal_value is not None:
    print(f"\n=== KẾT QUẢ SAU {iterations} VÒNG LẶP ===")
    print(f"Giá trị tối ưu: {optimal_value}")
    print("Nghiệm tối ưu:")
    for i, val in enumerate(optimal_solution):
        print(f"x{i+1} = {val}")
else:
    print("\nBài toán không có nghiệm khả thi hoặc không giới nội")


=== KẾT QUẢ SAU 3 VÒNG LẶP ===
Giá trị tối ưu: 81250.0
Nghiệm tối ưu:
x1 = 12.5
x2 = 25.0


In [11]:
# Visualization - Hiển thị trực quan các bước giải

def display_tableau_history(tableau_history, var_names):
    """
    Hiển thị lịch sử các bảng đơn hình.
    
    Args:
        tableau_history (list): Lịch sử các bảng đơn hình
        var_names (list): Tên của các biến
    """
    for step in tableau_history:
        display(Markdown(f"### Vòng lặp {step['iteration']}"))
        
        if step['pivot_row'] is not None and step['pivot_col'] is not None:
            display(Markdown(f"**Pivot:** Hàng = {step['pivot_row'] + 1}, Cột = {var_names[step['pivot_col']]}"))
        
        # Tạo DataFrame cho bảng đơn hình
        data = []
        for i in range(len(step['basis'])):
            row_data = {
                'Biến cơ sở': var_names[step['basis'][i]],
                'Hệ số': step['basis_cost'][i],
                'RHS': step['b'][i]
            }
            for j, name in enumerate(var_names):
                row_data[name] = step['tableau'][i][j]
            data.append(row_data)
        
        # Thêm hàng Z
        z_data = {
            'Biến cơ sở': 'Z',
            'Hệ số': '',
            'RHS': step['rhs_z']
        }
        for j, name in enumerate(var_names):
            z_data[name] = step['z_row'][j]
        data.append(z_data)
        
        # Tạo và hiển thị DataFrame
        df = pd.DataFrame(data)
        display(df)
        
        # Kiểm tra điều kiện dừng
        if step['iteration'] > 0:
            min_z_row = min(step['z_row'])
            if min_z_row >= 0:
                display(Markdown("**Điều kiện dừng:** Tất cả reduced costs ≥ 0, đã đạt tối ưu"))
        
        display(Markdown("---"))

# Hiển thị trực quan các bước giải
if tableau_history:
    basis, basis_cost, b, tableau, z_row, rhs_z, var_names, c_extended = build_bigM_tableau(c, A, b, signs)
    display_tableau_history(tableau_history, var_names)

### Vòng lặp 0

,Biến cơ sở,Hệ số,RHS,x1,x2,x3,x4,x5,x6
0,x3,0,40,1,1,1,0,0,0
1,x4,0,9000,150,250,0,1,0,0
2,x5,0,50000,1000,1500,0,0,1,0
3,x6,0,0,2,-1,0,0,0,1
4,Z,,0,-2500,-2000,0,0,0,0


---

### Vòng lặp 1

**Pivot:** Hàng = 4, Cột = x1

...

### Vòng lặp 2

**Pivot:** Hàng = 3, Cột = x2

,Biến cơ sở,Hệ số,RHS,x1,x2,x3,x4,x5,x6
0,x3,0,2.5,0.0,0.0,1.0,0.0,-0.00075,-0.125
1,x4,0,875.0,0.0,0.0,0.0,1.0,-0.16250,6.250
2,x2,2000,25.0,0.0,1.0,0.0,0.0,0.00050,-0.250
3,x1,2500,12.5,1.0,0.0,0.0,0.0,0.00025,0.375
4,Z,,81250.0,0.0,0.0,0.0,0.0,1.62500,437.500


**Điều kiện dừng:** Tất cả reduced costs ≥ 0, đã đạt tối ưu

---

In [ ]:
# Phân tích nghiệm tối ưu và biến số đối ngẫu

def analyze_solution(optimal_solution, optimal_value, tableau_history, var_names, c):
    """
    Phân tích nghiệm tối ưu và các thông tin bổ sung.
    
    Args:
        optimal_solution (list): Nghiệm tối ưu
        optimal_value (float): Giá trị tối ưu
        tableau_history (list): Lịch sử các bảng đơn hình
        var_names (list): Tên của các biến
        c (list): Hệ số hàm mục tiêu
    """
    if optimal_solution is None:
        display(Markdown("## Kết quả: Bài toán không có nghiệm khả thi hoặc không giới nội"))
        return
    
    # Thông tin nghiệm tối ưu
    display(Markdown("## Kết quả tối ưu"))
    display(Markdown(f"**Giá trị tối ưu:** Z = {optimal_value}"))
    
    # Giá trị các biến
    display(Markdown("### Giá trị các biến quyết định"))
    sol_data = {}
    for i, val in enumerate(optimal_solution):
        sol_data[f"x{i+1}"] = val
    sol_df = pd.DataFrame([sol_data])
    display(sol_df)
    
    # Phân tích biến số đối ngẫu (shadow prices)
    last_step = tableau_history[-1]
    num_original_vars = len(c)
    
    # Tìm các biến slack/surplus/artificial và shadow prices
    display(Markdown("### Phân tích độ nhạy"))
    shadow_prices = []
    
    for j in range(num_original_vars, len(var_names)):
        # Tìm reduced cost cho biến không cơ sở
        if j not in last_step['basis']:
            shadow_prices.append((var_names[j], last_step['z_row'][j]))